# 回帰における評価指標

In [1]:
import numpy as np
import pandas as pd

## RMSE (Root Mean Squared Error)
$$
{\large \rm{RMSE} =  \sqrt{\frac{1}{N}\sum_{i=1}^{N}(y_i-\hat{y}_i)^2}}
$$

In [2]:
from sklearn.metrics import mean_squared_error

y_true = [1.0, 1.5, 2.0, 1.2, 1.8]
y_pred = [0.8, 1.5, 1.8, 1.3, 3.0]

rmse = np.sqrt(mean_squared_error(y_true, y_pred))
print(rmse)

0.5531726674375732


## RMSLE (Root Mean Squared Logarithmic Error)
$$
{\large \rm{RMSLE} = \sqrt{\frac{1}{N}\sum_{i=1}^{N}\{\log(1+y_i)-\log(1+\hat{y}_i)\}^2}}
$$

- 目的変数が裾の長い分布の持ち、大きな値の影響が強いときに使う
- 真の値が０のときのために1加えてから対数をとる

## MAE (Mean Absolute Error)
$$
{\large \rm{RAE} = \frac{1}{N}\sum_{i=1}^{N}|y_i-\hat{y}_i|}
$$

- 外れ値の影響を低減した評価に適している
- 数学的に扱いづらい

## 決定係数 ($R^2$)
$$
{\large R^2 = 1 - \frac{\sum_{i=1}^{N}(y_i-\hat{y}_i)^2}{\sum_{i=1}^{N}(y_i-\bar{y}_i)^2}}
$$

- 回帰分析の当てはまりの良さを表す
- 分母は定数なので$R^2$の最大化は分子のRMSEの最小化と同値

# 二値分類における評価指標
## 準備：混同行列 (confusion matrix)
- TP (True Positive)：予測値を正例として、その予測値が正しい場合
- TN (True Negative)：予測値を負例として、その予測値が正しい場合
- FP (False Positive)：予測値を正例として、その予測値が誤りの場合
- FN (False Negative)：予測値を負例としてその予測値が誤りの場合

In [3]:
from sklearn.metrics import confusion_matrix

y_true = [1, 0, 1, 1, 0, 1, 1, 0]
y_pred = [0, 0, 1, 1, 0, 0, 1, 1]

tp = np.sum((np.array(y_true) == 1) & (np.array(y_pred) == 1))
tn = np.sum((np.array(y_true) == 0) & (np.array(y_pred) == 0))
fp = np.sum((np.array(y_true) == 0) & (np.array(y_pred) == 1))
fn = np.sum((np.array(y_true) == 1) & (np.array(y_pred) == 0))

confusion_matrix = np.array([[tp, fp],
                             [fn, tn]])
print(confusion_matrix)

[[3 1]
 [2 2]]


## accuracy（正答率）とerror rate（誤答率）
$$
{\large {\rm accuracy} = \frac{TP+TN}{TP + TN+FP+FN}}
$$

$$
{\large {\rm error rate} = 1 - {\rm accuracy}}
$$

- 不均衡なデータではモデルの評価をしづらい

In [4]:
from sklearn.metrics import accuracy_score

y_true = [1, 0, 1, 1, 0, 1, 1, 0]
y_pred = [0, 0, 1, 1, 0, 0, 1, 1]

accuracy = accuracy_score(y_true, y_pred)
print(accuracy)

0.625


## precision（適合率）とrecall（再現率）
$$
{\large {\rm precision} = \frac{TP}{TP+FP}}
$$

$$
{\large {\rm recall} = \frac{TP}{TP+FN}}
$$

- いずれも１に近づく方が良いスコア
- precisionとrecallは互いにトレードオフの関係
    - 単体では指標とはならず、セットで指標となる
- 誤検知を少なくした場合はprecisionを重視
- 正例の見逃しを避けたい場合はrecallを重視

## F1-scoreとFβ-score
$$
{\large {\rm F_1} = \frac{2}{\frac{1}{\rm recall}+\frac{1}{\rm precision}}
= \frac{2\cdot{\rm recall}\cdot{\rm precision}}{{\rm recall}+ {\rm precision}}
= \frac{2TP}{2TP+FP+FN}}
$$

$$
{\large {\rm F_{\beta}} = \frac{1+\beta^2}{\frac{\beta^2}{\rm recall}+\frac{1}{\rm precision}}
= \frac{(1+\beta^2)\cdot{\rm recall}\cdot{\rm precision}}{{\rm recall}+ \beta^2{\rm precision}}}
$$

- F1-scoreはprecisionとrecallの調和平均でよく使われる
- F1-scoreはF値とも呼ばれる
- Fβ-scoreはrecallをどれだけ重視するかを係数βによって調整した指標

## MCC (Matthews Correlation Coefficient)
$$
{\large {\rm MCC} = \frac{TP \times TN - FP \times FN}{\sqrt{(TP+FP)(TP+FN)(TN+FP)(TN+FN)}}}
$$

- 不均衡なデータに対してモデルの性能を適切に評価しやすい
- -1から+1の範囲の値をとり、+1のときに完璧な予測、0のときにランダムな予測、-1のときに完全に反対の予測をしていることになる

## logloss
$$
{\large {\rm logloss} = -\frac{1}{N} \sum_{i=1}^{N}(y_i\log p_i+(1-y_i)\log(1-p_1))
= - \frac{1}{N}\sum_{i=1}^{N}\log p'_i
}
$$

- 正例である確率を予測値とする場合の評価指標
- $y_i$は正例かどうかを表すラベル
- $p_i$は各レコードが正例である予測確率
- $p'_i$は真の予測をしている確率
- 低い方が良い指標
- 正例である確率を低く予測したにも関わらず正例である場合やその逆の場合はペナルティが大きく与えられる

In [5]:
from sklearn.metrics import log_loss

y_true = [1, 0, 1, 1, 0, 1]
y_pred = [0.1, 0.2, 0.8, 0.8, 0.1, 0.3]

logloss = log_loss(y_true, y_pred)
print(logloss)

0.7135581778200728
